# Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib notebook
import sys
###
#!pip install openslide-python
#https://github.com/deroneriksson/python-wsi-preprocessing/
###
sys.path.append('../python-wsi-preprocessing/')
from deephistopath.wsi import slide, filter, tiles, util

import fastai
from fastai.vision import *
from fastai.core import parallel

import pandas as pd
import numpy as np
import os
from pathlib import Path
from functools import partial, update_wrapper
from tqdm import tqdm_notebook as tqdm
from enum import Enum
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2
import openslide
from openslide.lowlevel import *
Image.MAX_IMAGE_PIXELS = 10000000000

PATH = Path('/home/Deep_Learner/private/network/datasets/Hypophysenadenome/')
FONT_PATH=PATH/'1984-Happines-Regular.ttf'
ROIS_CORTICOTROP = PATH/'rois_corticotrop'
ROIS_CORTICOTROP_FILTERED = PATH/'rois_corticotrop_filtered'
ROIS_GONADOTROP = PATH/'rois_gonadotrop'
ROIS_GONADOTROP_FILTERED = PATH/'rois_gonadotrop_filtered'
TILES_CORTICOTROP = PATH/'tiles_corticotrop'
TILES_GONADOTROP = PATH/'tiles_gonadotrop'

ROIS_EXPERIMENTING = PATH/'rois_experimenting'
ROIS_EXPERIMENTING_FILTERED = PATH/'rois_experimenting_filtered'
TILES_EXPERIMENTING = PATH/'tiles_experimenting'

PATH_RELAPSE = Path('/home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/')
RELAPSE_WSIS_EXPERIMENTING = PATH_RELAPSE/'wsis_experimenting'
RELAPSE_IMAGES_EXPERIMENTING = PATH_RELAPSE/'imgs_experimenting'
RELAPSE_IMAGES_FILTERED_EXPERIMENTING = PATH_RELAPSE/'imgs_filtered_experimenting'
RELAPSE_TILES_EXPERIMENTING = PATH_RELAPSE/'tiles_experimenting'

NONE_RELAPSE_WSIS = PATH_RELAPSE/'wsis_non_relapse'
NONE_RELAPSE_IMAGES = PATH_RELAPSE/'imgs_non_relapse'
NONE_RELAPSE_IMAGES_FILTERED = PATH_RELAPSE/'imgs_filtered_non_relapse'
NONE_RELAPSE_TILES = PATH_RELAPSE/'tiles_non_relapse'

RELAPSE_WSIS = PATH_RELAPSE/'wsis_relapse'
RELAPSE_IMAGES = PATH_RELAPSE/'imgs_relapse'
RELAPSE_IMAGES_FILTERED = PATH_RELAPSE/'imgs_filtered_relapse'
RELAPSE_TILES = PATH_RELAPSE/'tiles_relapse'


def show(np):
    return util.np_to_pil(np)

Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]

def show_multiple_images(path, rows = 3, figsize=(128, 64)):
    imgs = [open_image(p) for p in path.ls()]
    show_all(imgs=imgs, r=rows, figsize=figsize)
    
def show_multiple_images_big(path:pathlib.Path):
    for p in path.ls():
        plt.imshow(mpimg.imread(str(p)))
        plt.show()
        
class AdenomaType(Enum):
    experimenting = 0
    corticotrop = 1
    gonadotrop = 2
    relapse_experimenting = 3
    relapse = 4
    non_relapse = 5

In [2]:
adenomaType = AdenomaType.non_relapse

if adenomaType == AdenomaType.gonadotrop:
    rois_filtered_path = ROIS_GONADOTROP_FILTERED
    rois_path = ROIS_GONADOTROP
    tiles_path = TILES_GONADOTROP
    
if adenomaType == AdenomaType.corticotrop:
    rois_filtered_path = ROIS_CORTICOTROP_FILTERED
    rois_path = ROIS_CORTICOTROP
    tiles_path = TILES_CORTICOTROP
    
if adenomaType == AdenomaType.experimenting:
    wsi_path = ROIS_EXPERIMENTING
    rois_filtered_path = ROIS_EXPERIMENTING_FILTERED
    rois_path = ROIS_EXPERIMENTING
    tiles_path = TILES_EXPERIMENTING
    
if adenomaType == AdenomaType.relapse_experimenting:
    wsi_path = RELAPSE_WSIS_EXPERIMENTING
    rois_filtered_path = RELAPSE_IMAGES_FILTERED_EXPERIMENTING
    rois_path = RELAPSE_IMAGES_EXPERIMENTING
    tiles_path = RELAPSE_TILES_EXPERIMENTING
    
if adenomaType == AdenomaType.relapse:
    wsi_path = RELAPSE_WSIS
    rois_path = RELAPSE_IMAGES
    rois_filtered_path = RELAPSE_IMAGES_FILTERED
    tiles_path = RELAPSE_TILES  
    
if adenomaType == AdenomaType.non_relapse:
    wsi_path = NONE_RELAPSE_WSIS
    rois_path = NONE_RELAPSE_IMAGES
    rois_filtered_path = NONE_RELAPSE_IMAGES_FILTERED
    tiles_path = NONE_RELAPSE_TILES

In [3]:
rois_path.mkdir(exist_ok=True)
rois_filtered_path.mkdir(exist_ok=True)
tiles_path.mkdir(exist_ok=True)

# Overwrite parts of wsi lib

In [4]:
slide.SCALE_FACTOR = 2


slide.BASE_DIR = PATH/'data'
slide.SRC_TRAIN_EXT = "ndpi"
tiles.SUMMARY_TITLE_FONT_PATH = str(FONT_PATH)
tiles.FONT_PATH = str(FONT_PATH)
tiles.DISPLAY_TILE_SUMMARY_LABELS = True
tiles.LABEL_ALL_TILES_IN_TOP_TILE_SUMMARY = True
tiles.BORDER_ALL_TILES_IN_TOP_TILE_SUMMARY = True
tiles.TISSUE_LOW_THRESH = 20

tiles.ROW_TILE_SIZE = 1024
tiles.COL_TILE_SIZE = 1024

##################################################################################################################

slide.open_slide = slide.open_image

##################################################################################################################

def get_image_path(folder_path, slide_num):
    return folder_path.ls()[slide_num]

slide.get_filter_image_result = partial(get_image_path, rois_filtered_path)
slide.get_training_image_path = partial(get_image_path, rois_path)
slide.get_training_slide_path = partial(get_image_path, wsi_path)

##################################################################################################################

def parse_dimensions_from_image_filename(img_path):
    shape = slide.open_image_np(img_path).shape
    return shape[1], shape[0], shape[1], shape[0]

slide.parse_dimensions_from_image_filename = parse_dimensions_from_image_filename

##################################################################################################################

def tile_to_pil_tile(tile):
  """
  Convert tile information into the corresponding tile as a PIL image read from the whole-slide image file.

  Args:
    tile: Tile object.

  Return:
    Tile as a PIL image.
  """
  t = tile
  filepath = slide.get_training_image_path(t.slide_num)
  img = slide.open_image_np(filepath)
  #x, y = t.o_c_s, t.o_r_s
  #w, h = t.o_c_e - t.o_c_s, t.o_r_e - t.o_r_s
  tile = img[int(t.o_r_s/slide.SCALE_FACTOR):int(t.o_r_e/slide.SCALE_FACTOR), int(t.o_c_s/slide.SCALE_FACTOR):int(t.o_c_e/slide.SCALE_FACTOR),:]
  tile_pil = util.np_to_pil(tile)
  return tile_pil

tiles.tile_to_pil_tile = tile_to_pil_tile

##################################################################################################################
def get_tile_image_path(tile):
  """
  Obtain tile image path based on tile information such as row, column, row pixel position, column pixel position,
  pixel width, and pixel height.

  Args:
    tile: Tile object.

  Returns:
    Path to image tile.
  """
  t = tile
  roi_name = slide.get_training_image_path(t.slide_num).stem
  tile_path = os.path.join(PATH, tiles_path,
                           roi_name + "-" + 'tile' + "-r%d-c%d-x%d-y%d-w%d-h%d" % (
                             t.r, t.c, t.o_c_s, t.o_r_s, t.o_c_e - t.o_c_s, t.o_r_e - t.o_r_s) + "." + 'png')
  return tile_path

slide.get_tile_image_path = get_tile_image_path

##################################################################################################################

def top_tiles(self):
    """
    Retrieve the top-scoring tiles.

    Returns:
       List of the top-scoring tiles.
    """
    sorted_tiles = self.tiles_by_score()
    top_tiles = [tile for tile in sorted_tiles
                 if check_tile(tile)]
    print(f'Number of top tiles/all tiles: {len(top_tiles)}/{len(sorted_tiles)}')
    return top_tiles

def check_tile(tile):
    width = tile.o_c_e - tile.o_c_s
    height = tile.o_r_e - tile.o_r_s
    return tile.score > 0.55 and width >= 0.7*tiles.COL_TILE_SIZE and height >= 0.7*tiles.ROW_TILE_SIZE


tiles.TileSummary.top_tiles = top_tiles

##################################################################################################################
from deephistopath.wsi.tiles import (hsv_saturation_and_value_factor, 
                                     hsv_purple_pink_factor, 
                                     tissue_quantity_factor, 
                                     tissue_quantity)

def score_tile(np_tile, tissue_percent, slide_num, row, col):
  """
  Score tile based on tissue percentage, color factor, saturation/value factor, and tissue quantity factor.

  Args:
    np_tile: Tile as NumPy array.
    tissue_percent: The percentage of the tile judged to be tissue.
    slide_num: Slide number.
    row: Tile row.
    col: Tile column.

  Returns tuple consisting of score, color factor, saturation/value factor, and tissue quantity factor.
  """
  color_factor = hsv_purple_pink_factor(np_tile)
  s_and_v_factor = hsv_saturation_and_value_factor(np_tile)
  amount = tissue_quantity(tissue_percent)
  quantity_factor = tissue_quantity_factor(amount)
  combined_factor = color_factor * s_and_v_factor
  score = tissue_percent * combined_factor / 1000.0
  # scale score to between 0 and 1
  score = 1.0 - (10.0 / (10.0 + score))
  #print(f'tp: {tissue_percent}')
  #print(f'cf: {combined_factor}')
  #print(f'score: {score}')  
  return score, color_factor, s_and_v_factor, quantity_factor

tiles.score_tile = score_tile
#tiles.score_tile = tiles.score_tile
#############################################################################################################

def _load_image(buf, size):
        '''buf must be a buffer.'''

        # Load entire buffer at once if possible
        MAX_PIXELS_PER_LOAD = (1 << 29) - 1
        # Otherwise, use chunks smaller than the maximum to reduce memory
        # requirements
        PIXELS_PER_LOAD = 1 << 26

        def do_load(buf, size):
            '''buf can be a string, but should be a ctypes buffer to avoid an
            extra copy in the caller.'''
            # First reorder the bytes in a pixel from native-endian aRGB to
            # big-endian RGBa to work around limitations in RGBa loader
            rawmode = (sys.byteorder == 'little') and 'BGRA' or 'ARGB'
            buf = PIL.Image.frombuffer('RGBA', size, buf, 'raw', rawmode, 0, 1)
            # Image.tobytes() is named tostring() in Pillow 1.x and PIL
            buf = (getattr(buf, 'tobytes', None) or buf.tostring)()
            # Now load the image as RGBA, undoing premultiplication
            return PIL.Image.frombuffer('RGBA', size, buf, 'raw', 'RGBa', 0, 1)

        # Fast path for small buffers
        w, h = size
        if w * h <= MAX_PIXELS_PER_LOAD:
            return do_load(buf, size)

        # Load in chunks to avoid OverflowError in PIL.Image.frombuffer()
        # https://github.com/python-pillow/Pillow/issues/1475
        if w > PIXELS_PER_LOAD:
            # We could support this, but it seems like overkill
            raise ValueError('Width %d is too large (maximum %d)' %
                    (w, PIXELS_PER_LOAD))
        rows_per_load = PIXELS_PER_LOAD // w
        img = PIL.Image.new('RGBA', (w, h))
        for y in range(0, h, rows_per_load):
            rows = min(h - y, rows_per_load)
            if sys.version[0] == '2':
                chunk = buffer(buf, 4 * y * w, 4 * rows * w)
            else:
                # PIL.Image.frombuffer() won't take a memoryview or
                # bytearray, so we can't avoid copying
                chunk = memoryview(buf)[y * w:(y + rows) * w].tobytes()
            img.paste(do_load(chunk, (w, rows)), (0, y))
        return img

openslide.lowlevel._load_image = _load_image


def slide_to_scaled_pil_image(slide_filepath):
  """
  Convert a WSI training slide to a scaled-down PIL image.

  Args:
    slide_number: The slide number.

  Returns:
    Tuple consisting of scaled-down PIL image, original width, original height, new width, and new height.
  """
  sl = openslide.open_slide(str(slide_filepath))

  large_w, large_h = sl.dimensions
  new_w = math.floor(large_w / slide.SCALE_FACTOR)
  new_h = math.floor(large_h / slide.SCALE_FACTOR)
  level = sl.get_best_level_for_downsample(slide.SCALE_FACTOR)
  whole_slide_image = sl.read_region((0, 0), level, sl.level_dimensions[level])
  whole_slide_image = whole_slide_image.convert("RGB")
  img = whole_slide_image.resize((new_w, new_h), PIL.Image.BILINEAR)
  return img, large_w, large_h, new_w, new_h

slide.slide_to_scaled_pil_image = slide_to_scaled_pil_image


def training_slide_to_image(slide_filepath):
  """
  Convert a WSI training slide to a saved scaled-down image in a format such as jpg or png.

  Args:
    slide_number: The slide number.
  """
  img, large_w, large_h, new_w, new_h = slide_to_scaled_pil_image(slide_filepath)
  img_path = f'{rois_path}/{slide_filepath.stem}-scale_factor_{slide.SCALE_FACTOR}.png'
  print("Saving image to: " + img_path)
  img.save(img_path)

slide.training_slide_to_image = training_slide_to_image

##############################################################################################

def summary_and_tiles(slide_num, display=True, save_summary=False, save_data=True, save_top_tiles=True):
  """
  Generate tile summary and top tiles for slide.

  Args:
    slide_num: The slide number.
    display: If True, display tile summary to screen.
    save_summary: If True, save tile summary images.
    save_data: If True, save tile data to csv file.
    save_top_tiles: If True, save top tiles to files.

  """
  img_path = slide.get_filter_image_result(slide_num)
  np_img = slide.open_image_np(img_path)

  tile_sum = tiles.score_tiles(slide_num, np_img)
  if save_data:
    tiles.save_tile_data(tile_sum)
  #generate_tile_summaries(tile_sum, np_img, display=display, save_summary=save_summary)
  #generate_top_tile_summaries(tile_sum, np_img, display=display, save_summary=save_summary)
  if save_top_tiles:
    for tile in tile_sum.top_tiles():
      tile.save_tile()
  return tile_sum

tiles.summary_and_tiles = summary_and_tiles

# Convert WSIs

In [5]:
#def convert_wsi_to_png(path:pathlib.Path, index:int):
#    #try:
#        slide.training_slide_to_image(path)
#        print(f'Saved {path.stem}.png')
#    #except:
#        #print(path)
#        
#fastai.core.parallel(convert_wsi_to_png, wsi_path.ls(), max_workers=20)

In [ ]:
for p in tqdm(wsi_path.ls()):
    for b in rois_path.ls():
        if p.stem == b.stem:
            break
    else:
        slide.training_slide_to_image(p)

# Filter images

In [5]:
##
# for 'normal' img formats like .png
# fastai's parallel function expects a function that takes a value and an index, for that reason 
# filter_roi accepts the 'index' parameter, it can be ignored
##

def filter_roi(img_path:pathlib.Path, index:int = 0):
    if img_path.suffix == '.png':
        try:
            img_pil = slide.open_image(img_path)
            img_np = util.pil_to_np_rgb(img_pil)
            grayscale_np = filter.filter_rgb_to_grayscale(img_np)
            complement_np = filter.filter_complement(grayscale_np)
            otsu_np = filter.filter_otsu_threshold(complement_np).astype(np.bool)
            filtered_img_np = util.mask_rgb(img_np, otsu_np)
            #filtered_path = rois_filtered_path/f'{img_path.stem}-filtered{img_path.suffix}'
            filtered_path = rois_filtered_path/f'{img_path.stem}{img_path.suffix}'
            try:
                filtered_img_pil = util.np_to_pil(filtered_img_np)
                filtered_img_pil.save(filtered_path)
            except ValueError as e:
                print(f'PIL failed, using cv2 instead. {img_path}')
                #cv2 expects the array to have bgr as channel order
                bgr = filtered_img_np[...,[2,1,0]]
                cv2.imwrite(str(filtered_path), bgr)
        except:
            print(f'The error occured during the filtering process or PIL and cv2 failed to save it. {img_path}')
            raise
###
# use this line instead of the following for loop to do the filtering in parallel with mutiple threads
###
#parallel(filter_roi, rois_path.ls(), max_workers=25)

#failed = []
for p in tqdm(rois_path.ls()):
    #try:
        filter_roi(p)
    #except:
        #failed.append(p)
    
#print(failed)

RGB                  | Time: 0:00:07.352323  Type: uint8   Shape: (16128, 14336, 3)
Gray                 | Time: 0:00:11.113000  Type: uint8   Shape: (16128, 14336)
Complement           | Time: 0:00:00.070322  Type: uint8   Shape: (16128, 14336)
Otsu Threshold       | Time: 0:00:02.144515  Type: uint8   Shape: (16128, 14336)
Mask RGB             | Time: 0:00:00.794553  Type: uint8   Shape: (16128, 14336, 3)
RGB                  | Time: 0:00:12.326676  Type: uint8   Shape: (21504, 17920, 3)
Gray                 | Time: 0:00:18.222186  Type: uint8   Shape: (21504, 17920)
Complement           | Time: 0:00:00.111731  Type: uint8   Shape: (21504, 17920)
Otsu Threshold       | Time: 0:00:03.484084  Type: uint8   Shape: (21504, 17920)
Mask RGB             | Time: 0:00:01.300631  Type: uint8   Shape: (21504, 17920, 3)
RGB                  | Time: 0:01:26.960319  Type: uint8   Shape: (45696, 59136, 3)
Gray                 | Time: 0:02:08.530854  Type: uint8   Shape: (45696, 59136)
Complement   

PIL failed, using cv2 instead. /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/imgs_non_relapse/711-12-III-HE-scale_factor_2.png
RGB                  | Time: 0:00:14.969793  Type: uint8   Shape: (18816, 17920, 3)
Gray                 | Time: 0:00:19.668405  Type: uint8   Shape: (18816, 17920)
Complement           | Time: 0:00:00.137808  Type: uint8   Shape: (18816, 17920)
Otsu Threshold       | Time: 0:00:03.390516  Type: uint8   Shape: (18816, 17920)
Mask RGB             | Time: 0:00:01.335997  Type: uint8   Shape: (18816, 17920, 3)
RGB                  | Time: 0:00:17.707281  Type: uint8   Shape: (18816, 25088, 3)
Gray                 | Time: 0:00:27.483334  Type: uint8   Shape: (18816, 25088)
Complement           | Time: 0:00:00.180145  Type: uint8   Shape: (18816, 25088)
Otsu Threshold       | Time: 0:00:04.677538  Type: uint8   Shape: (18816, 25088)
Mask RGB             | Time: 0:00:01.870887  Type: uint8   Shape: (18816, 25088, 3)
RGB                  | Tim

RGB                  | Time: 0:00:08.874318  Type: uint8   Shape: (13440, 16128, 3)
Gray                 | Time: 0:00:12.714951  Type: uint8   Shape: (13440, 16128)
Complement           | Time: 0:00:00.085190  Type: uint8   Shape: (13440, 16128)
Otsu Threshold       | Time: 0:00:02.205660  Type: uint8   Shape: (13440, 16128)
Mask RGB             | Time: 0:00:00.898730  Type: uint8   Shape: (13440, 16128, 3)
RGB                  | Time: 0:00:19.893420  Type: uint8   Shape: (21504, 23296, 3)
Gray                 | Time: 0:00:29.104551  Type: uint8   Shape: (21504, 23296)
Complement           | Time: 0:00:00.185045  Type: uint8   Shape: (21504, 23296)
Otsu Threshold       | Time: 0:00:05.045240  Type: uint8   Shape: (21504, 23296)
Mask RGB             | Time: 0:00:02.055000  Type: uint8   Shape: (21504, 23296, 3)
RGB                  | Time: 0:00:35.988558  Type: uint8   Shape: (26880, 34048, 3)
Gray                 | Time: 0:00:53.081588  Type: uint8   Shape: (26880, 34048)
Complement   

## Fix for PIL Value Error
PIL raises an ValueEerror, when using Image.fromarray() with very big numpy arrays.
In this case use cv2. Beware that you have to switch rgb to bgr in numpy array before saving.

In [ ]:
pa = Path('/home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/imgs_relapse/495-09-III-HE-scale_factor_2.png')

In [ ]:
img_pil = slide.open_image(pa)
img_np = util.pil_to_np_rgb(img_pil)
grayscale_np = filter.filter_rgb_to_grayscale(img_np)
complement_np = filter.filter_complement(grayscale_np)
otsu_np = filter.filter_otsu_threshold(complement_np).astype(np.bool)
filtered_img_np = util.mask_rgb(img_np, otsu_np)

In [ ]:
plt.imshow(filtered_img_np)

In [ ]:
filtered_path = rois_filtered_path/f'{pa.stem}{pa.suffix}'

In [ ]:
try:
    filtered_img_pil = util.np_to_pil(filtered_img_np)
    filtered_img_pil.save(filtered_path)
except ValueError as e:
    bgr = filtered_img_np[...,[2,1,0]]
    cv2.imwrite(str(filtered_path), bgr)

In [ ]:
ü = filtered_img_np[10000:12000,3500:5000,:]

In [ ]:
plt.imshow(ü)

In [ ]:
path_pil = rois_filtered_path/f'{pa.stem}-pil-{pa.suffix}'
filtered_img_pil = util.np_to_pil(ü)

In [ ]:
path_cv = rois_filtered_path/f'{pa.stem}-cv2-{pa.suffix}'
bgr = ü[...,[2,1,0]]
cv2.imwrite(str(path_cv), bgr)

In [ ]:
open_image(path_cv)

In [ ]:
open_image(path_pil)

# Create tiles

## multi process

In [ ]:
tiles.multiprocess_filtered_images_to_tiles(display=False, 
                                            save_summary=False, 
                                            save_data=False, 
                                            save_top_tiles=True,
                                            html=False, 
                                            image_num_list=list(range(0, len(rois_filtered_path.ls()))))

Generating tile summaries (multiprocess)

Number of processes: 6
Number of training images: 39
Task #1: Process slides [0, 1, 2, 3, 4, 5]
Task #2: Process slides [6, 7, 8, 9, 10, 11, 12]
Task #3: Process slides [13, 14, 15, 16, 17, 18]
Task #4: Process slides [19, 20, 21, 22, 23, 24, 25]
Task #5: Process slides [26, 27, 28, 29, 30, 31]
Task #6: Process slides [32, 33, 34, 35, 36, 37, 38]
RGB                  | Time: 0:00:09.138319  Type: uint8   Shape: (16128, 14336, 3)
RGB                  | Time: 0:00:13.881033  Type: uint8   Shape: (18816, 19712, 3)
RGB                  | Time: 0:00:09.506314  Type: uint8   Shape: (16128, 14336, 3)
RGB                  | Time: 0:00:19.818907  Type: uint8   Shape: (13440, 43008, 3)


/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:265: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:271: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:279: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]


RGB                  | Time: 0:00:25.042628  Type: uint8   Shape: (21504, 23296, 3)
RGB                  | Time: 0:00:20.284284  Type: uint8   Shape: (18816, 19712, 3)


/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:265: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:271: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:279: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]


RGB                  | Time: 0:00:40.324946  Type: uint8   Shape: (13440, 43008, 3)


...

RGB                  | Time: 0:03:10.564396  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.367165  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r38-c75-x75776-y37888-w1024-h1024.png
RGB                  | Time: 0:03:10.745090  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.134363  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r19-c54-x54272-y18432-w1024-h1024.png
RGB                  | Time: 0:03:10.502643  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.125812  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r83-c30-x29696-y83968-w1024-h1024.png
RGB                  | Time: 0:03:11.423972  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Ti

RGB                  | Time: 0:03:11.465263  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.119785  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r44-c27-x26624-y44032-w1024-h1024.png
RGB                  | Time: 0:03:11.178187  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.121405  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r47-c91-x92160-y47104-w1024-h1024.png
RGB                  | Time: 0:03:12.069245  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.293534  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r72-c60-x60416-y72704-w1024-h1024.png
RGB                  | Time: 0:03:10.514857  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Ti

RGB                  | Time: 0:03:11.888841  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.449833  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r34-c68-x68608-y33792-w1024-h1024.png
RGB                  | Time: 0:03:13.762402  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.243182  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r49-c99-x100352-y49152-w1024-h1024.png
RGB                  | Time: 0:03:12.464523  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.132731  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r50-c7-x6144-y50176-w1024-h1024.png
RGB                  | Time: 0:03:10.581858  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Tim

RGB                  | Time: 0:03:11.559496  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.120293  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r47-c101-x102400-y47104-w1024-h1024.png
RGB                  | Time: 0:03:12.186256  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.123653  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r51-c94-x95232-y51200-w1024-h1024.png
RGB                  | Time: 0:03:10.662015  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.500675  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r55-c30-x29696-y55296-w1024-h1024.png
RGB                  | Time: 0:03:12.779628  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | 

RGB                  | Time: 0:03:10.565073  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.120098  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r52-c30-x29696-y52224-w1024-h1024.png
RGB                  | Time: 0:03:10.579090  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.135550  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r24-c38-x37888-y23552-w1024-h1024.png
RGB                  | Time: 0:03:10.676899  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Time: 0:00:00.124262  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1340-12-III-HE-scale_factor_2-tile-r49-c86-x87040-y49152-w1024-h1024.png
RGB                  | Time: 0:03:11.195200  Type: uint8   Shape: (48384, 57344, 3)
Save Tile            | Ti

In [11]:
print(len(tiles_path.ls()))

def get_wsi_name_from_path(path):
    split = path.stem.split('-')
    return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}'

l = []
for p in tqdm(tiles_path.ls()):
    l.append(get_wsi_name_from_path(p))

print(len(set(l)))
print(set(l))

3320



14
{'400-13-III-HE', '711-12-III-HE', '671-12-III-HE', '1567-12-III-HE', '1583-13-III-HE', '670-12-III-HE', '1314-12-Z-HE', '865-13-III-HE', '510-13-III-HE', '1739-12-IV-HE', '1340-12-III-HE', '1787-12-V-HE', '848-13-III-HE', '788-13-III-HE'}


In [ ]:
show_multiple_images_big(tiles_path)

In [ ]:
#!rm -r {rois_filtered_path/'*'}
#!rm -r {tiles_path/'*'}

## single process

In [ ]:
#!rm -r {tiles_path/'*'}

In [8]:
failed = []
for n, p in tqdm(enumerate(rois_filtered_path.ls()), total=len(rois_filtered_path.ls())):
    if p.suffix == '.png':
        try:
            print(p)
            tiles.summary_and_tiles(n, display=False, save_summary=False, save_data=False, save_top_tiles=True)
        except:
            failed.append(p)
            
print(failed)

/home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/imgs_filtered_non_relapse/400-13-III-HE-scale_factor_2.png
RGB                  | Time: 0:00:04.247988  Type: uint8   Shape: (16128, 14336, 3)
RGB                  | Time: 0:00:04.128956  Type: uint8   Shape: (16128, 14336, 3)


/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:265: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:271: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:279: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]


RGB                  | Time: 0:00:07.410383  Type: uint8   Shape: (16128, 14336, 3)
RGB                  | Time: 0:00:07.297548  Type: uint8   Shape: (16128, 14336, 3)
Number of top tiles/all tiles: 23/896
RGB                  | Time: 0:00:00.653640  Type: uint8   Shape: (16684, 14336, 3)
RGB                  | Time: 0:00:00.000489  Type: uint8   Shape: (388, 491, 3)
RGB                  | Time: 0:00:00.653066  Type: uint8   Shape: (16684, 14336, 3)
RGB                  | Time: 0:00:00.000311  Type: uint8   Shape: (388, 491, 3)
Number of top tiles/all tiles: 23/896
RGB                  | Time: 0:00:06.935153  Type: uint8   Shape: (16128, 14336, 3)
Save Tile            | Time: 0:00:00.046021  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/400-13-III-HE-scale_factor_2-tile-r18-c12-x11264-y17408-w1024-h1024.png
RGB                  | Time: 0:00:06.937421  Type: uint8   Shape: (16128, 14336, 3)
Save Tile            | Time: 0:00:00.047572  Nam

Number of top tiles/all tiles: 65/1470
RGB                  | Time: 0:00:11.899160  Type: uint8   Shape: (21504, 17920, 3)
Save Tile            | Time: 0:00:00.058945  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1567-12-III-HE-scale_factor_2-tile-r33-c21-x20480-y32768-w1024-h1024.png
RGB                  | Time: 0:00:11.897507  Type: uint8   Shape: (21504, 17920, 3)
Save Tile            | Time: 0:00:00.052705  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1567-12-III-HE-scale_factor_2-tile-r33-c24-x23552-y32768-w1024-h1024.png
RGB                  | Time: 0:00:11.892857  Type: uint8   Shape: (21504, 17920, 3)
Save Tile            | Time: 0:00:00.060011  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1567-12-III-HE-scale_factor_2-tile-r36-c24-x23552-y35840-w1024-h1024.png
RGB                  | Time: 0:00:11.893280  Type: uint8   Shape: (215

RGB                  | Time: 0:00:11.873877  Type: uint8   Shape: (21504, 17920, 3)
Save Tile            | Time: 0:00:00.058069  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1567-12-III-HE-scale_factor_2-tile-r31-c21-x20480-y30720-w1024-h1024.png
RGB                  | Time: 0:00:11.871355  Type: uint8   Shape: (21504, 17920, 3)
Save Tile            | Time: 0:00:00.068323  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1567-12-III-HE-scale_factor_2-tile-r36-c25-x24576-y35840-w1024-h1024.png
RGB                  | Time: 0:00:11.868328  Type: uint8   Shape: (21504, 17920, 3)
Save Tile            | Time: 0:00:00.060996  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1567-12-III-HE-scale_factor_2-tile-r26-c12-x11264-y25600-w1024-h1024.png
RGB                  | Time: 0:00:11.866919  Type: uint8   Shape: (21504, 17920, 3)
Save Tile            | Ti

...

RGB                  | Time: 0:00:17.518841  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.050605  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r13-c19-x18432-y12288-w1024-h1024.png
RGB                  | Time: 0:00:17.534996  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.046058  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r29-c28-x27648-y28672-w1024-h1024.png
RGB                  | Time: 0:00:17.542152  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.049954  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r11-c9-x8192-y10240-w1024-h1024.png
RGB                  | Time: 0:00:17.460185  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0

RGB                  | Time: 0:00:17.441015  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.047717  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r25-c23-x22528-y24576-w1024-h1024.png
RGB                  | Time: 0:00:17.431781  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.061003  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r24-c21-x20480-y23552-w1024-h1024.png
RGB                  | Time: 0:00:17.429733  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.046709  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r7-c7-x6144-y6144-w1024-h1024.png
RGB                  | Time: 0:00:17.542103  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:0

RGB                  | Time: 0:00:17.434575  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.046612  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r18-c17-x16384-y17408-w1024-h1024.png
RGB                  | Time: 0:00:17.414325  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.052824  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r6-c29-x28672-y5120-w1024-h1024.png
RGB                  | Time: 0:00:17.263610  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0:00:00.052887  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1739-12-IV-HE-scale_factor_2-tile-r18-c15-x14336-y17408-w1024-h1024.png
RGB                  | Time: 0:00:17.459975  Type: uint8   Shape: (21504, 23296, 3)
Save Tile            | Time: 0

RGB                  | Time: 0:00:14.209859  Type: uint8   Shape: (21504, 30464, 3)
RGB                  | Time: 0:00:14.195521  Type: uint8   Shape: (21504, 30464, 3)
RGB                  | Time: 0:00:22.545364  Type: uint8   Shape: (21504, 30464, 3)
RGB                  | Time: 0:00:22.281062  Type: uint8   Shape: (21504, 30464, 3)
Number of top tiles/all tiles: 0/2520
RGB                  | Time: 0:00:01.898148  Type: uint8   Shape: (21804, 30720, 3)
RGB                  | Time: 0:00:00.000045  Type: uint8   Shape: (36, 152, 3)
RGB                  | Time: 0:00:01.835003  Type: uint8   Shape: (21804, 30720, 3)
RGB                  | Time: 0:00:00.000052  Type: uint8   Shape: (36, 152, 3)
Number of top tiles/all tiles: 0/2520
/home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/imgs_filtered_non_relapse/848-13-III-HE-scale_factor_2.png
RGB                  | Time: 0:00:04.297671  Type: uint8   Shape: (13440, 16128, 3)
RGB                  | Time: 0:00:04.288191  Typ

Process ForkPoolWorker-78:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


In [14]:
for n, p in tqdm(enumerate(rois_filtered_path.ls()), total=len(rois_filtered_path.ls())):
    if p.name == '1314-12-Z-HE-scale_factor_2.png':
        print(n)

2



In [12]:
pa = rois_filtered_path/'1314-12-Z-HE-scale_factor_2.png';pa

PosixPath('/home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/imgs_filtered_non_relapse/1314-12-Z-HE-scale_factor_2.png')

In [13]:
pa.name

'1314-12-Z-HE-scale_factor_2.png'

In [15]:
tiles.summary_and_tiles(2, display=False, save_summary=False, save_data=False, save_top_tiles=True)

RGB                  | Time: 0:01:09.977356  Type: uint8   Shape: (45696, 59136, 3)
RGB                  | Time: 0:00:59.367628  Type: uint8   Shape: (45696, 59136, 3)


/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:265: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:271: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:279: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]


ValueError: not enough image data

In [20]:
tiles.summary_and_tiles(2, display=False, save_summary=False, save_data=False, save_top_tiles=True)

RGB                  | Time: 0:01:00.387613  Type: uint8   Shape: (45696, 59136, 3)
RGB                  | Time: 0:01:00.606001  Type: uint8   Shape: (45696, 59136, 3)


/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:265: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:271: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/opt/conda/lib/python3.7/site-packages/skimage/color/colorconv.py:279: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]


Number of top tiles/all tiles: 1966/10440
RGB                  | Time: 0:01:46.858399  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.098696  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r57-c50-x50176-y57344-w1024-h1024.png
RGB                  | Time: 0:01:45.021292  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.098719  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r71-c50-x50176-y71680-w1024-h1024.png
RGB                  | Time: 0:01:44.704975  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.050556  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r34-c90-x91136-y33792-w1024-h1024.png
RGB                  | Time: 0:01:43.208032  Type: uint8   Shape: (45696,

RGB                  | Time: 0:01:51.161586  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.061500  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r33-c90-x91136-y32768-w1024-h1024.png
RGB                  | Time: 0:01:43.598348  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.062815  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r79-c77-x77824-y79872-w1024-h1024.png
RGB                  | Time: 0:01:45.090539  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.065922  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r45-c98-x99328-y45056-w1024-h1024.png
RGB                  | Time: 0:01:45.041455  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:

RGB                  | Time: 0:01:33.493287  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.061178  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r52-c92-x93184-y52224-w1024-h1024.png
RGB                  | Time: 0:01:32.226222  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.068969  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r36-c88-x89088-y35840-w1024-h1024.png
RGB                  | Time: 0:01:32.879453  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.071604  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r47-c102-x103424-y47104-w1024-h1024.png
RGB                  | Time: 0:01:32.917068  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 

RGB                  | Time: 0:01:32.713351  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.055225  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r33-c28-x27648-y32768-w1024-h1024.png
RGB                  | Time: 0:01:32.916758  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.053510  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r45-c101-x102400-y45056-w1024-h1024.png
RGB                  | Time: 0:01:32.441030  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.053949  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r53-c86-x87040-y53248-w1024-h1024.png
RGB                  | Time: 0:01:32.626751  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 

RGB                  | Time: 0:01:32.370342  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.066174  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r57-c91-x92160-y57344-w1024-h1024.png
RGB                  | Time: 0:01:32.361398  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.055877  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r45-c50-x50176-y45056-w1024-h1024.png
RGB                  | Time: 0:01:32.076659  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:00:00.052306  Name: /home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/tiles_non_relapse/1314-12-Z-HE-scale_factor_2-tile-r31-c82-x82944-y30720-w1024-h1024.png
RGB                  | Time: 0:01:43.093732  Type: uint8   Shape: (45696, 59136, 3)
Save Tile            | Time: 0:

IndexError: too many indices for array

# exp

In [ ]:
import PIL
p = '/home/Deep_Learner/work/network/datasets/Hypophysenadenome/rois_corticotrop/1000-13-III-HE-ROI_1-ACTH.png'

img_pil = slide.open_image(p)
img_np = util.pil_to_np_rgb(img_pil)
grayscale_np = filter.filter_rgb_to_grayscale(img_np)
complement_np = filter.filter_complement(grayscale_np)
otsu_np = filter.filter_otsu_threshold(complement_np).astype(np.bool)
filtered_img_np = util.mask_rgb(img_np, otsu_np)

plt.imshow(filtered_img_np)

tiles = []
sz = 512
for i in range(int(filtered_img_np.shape[0]/sz)):
    for j in range(int(filtered_img_np.shape[1]/sz)):
         tiles.append(filtered_img_np[i*sz:(i+1)*sz,j*sz:(j+1)*sz])
        

for t in tiles:
    print(t.shape)
    nz = np.count_nonzero(t)// 3
    print(f'tissue percentage:{(nz/(t.shape[0]*t.shape[1]))*100}')
    plt.imshow(t)
    plt.show()